ksekinisame apo to baseline
meta skeftika na valo ta idia pou eixe gia abstracts kai gia tous authors
tora 8a prospa8iso na kano hyperparameter tuning kai na xrisimopoiiso ki alla modela
Meta 8a prospa8iso na xrisimopoiiso cosine similrity me tin xrisi to word2vec
to cosine similarity den doulevei giati ta vector exoun diaforetika sized kati prepei na allakso



In [86]:
import networkx as nx
import gensim
import csv
import numpy as np
from random import randint
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import nltk
from gensim.models import Word2Vec
from gensim.models import Doc2Vec

In [47]:
# Create a graph
G = nx.read_edgelist('edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 138499
Number of edges: 1091955


In [48]:
# Read the abstract of each paper
abstracts = dict()
with open('abstracts.txt', 'r', encoding='utf-8') as f:
    for line in f:
        node, abstract = line.split('|--|')
        abstracts[int(node)] = abstract

In [49]:
# Read the authors of each paper
authors = dict()
with open('authors.txt', 'r', encoding='utf-8') as f:
    for line in f:
        node, author = line.split('|--|')
        authors[int(node)] = author

In [50]:
# Map text to set of terms
for node in abstracts:
    abstracts[node] = set(abstracts[node].split())

In [51]:
# Map authors to set of terms
for node in authors:
    authors[node] = set(authors[node].split())

In [94]:
# Make all abstracts have the same length
model2 = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)

In [95]:
# create tagged documents from the abstracts
documents = []
for node in abstracts:
    documents.append(gensim.models.doc2vec.TaggedDocument(abstracts[node], [node]))

In [96]:
model2.build_vocab(documents)

In [99]:
model2.train(documents, total_examples=model2.corpus_count, epochs=model2.epochs)

In [73]:
import gensim.models.keyedvectors as word2vec
model = word2vec.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [77]:
# its class label is 1 if it corresponds to an edge and 0, otherwise.
# Use the following 3 features for each pair of nodes:
# (1) sum of number of unique terms of the two nodes' abstracts
# (2) absolute value of difference of number of unique terms of the two nodes' abstracts
# (3) number of common terms between the abstracts of the two nodes
# (4) sum of number of unique terms of the two nodes' authors
# (5) absolute value of difference of number of unique terms of the two nodes' authors
# (6) number of common terms between the authors of the two nodes
# (7) cosine similarity between the abstracts of the two nodes
X_train = np.zeros((2*m, 7))
y_train = np.zeros(2*m)
n = G.number_of_nodes()
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[i,0] = len(abstracts[edge[0]]) + len(abstracts[edge[1]])
    X_train[i,1] = abs(len(abstracts[edge[0]]) - len(abstracts[edge[1]]))
    X_train[i,2] = len(abstracts[edge[0]].intersection(abstracts[edge[1]]))
    X_train[i,3] = len(authors[edge[0]]) + len(authors[edge[1]])
    X_train[i,4] = abs(len(authors[edge[0]]) - len(authors[edge[1]]))
    X_train[i,5] = len(authors[edge[0]].intersection(authors[edge[1]]))
    X_train[i,6] = model[abstracts[edge[0]], abstracts[edge[1]]]
    y_train[i] = 1

    # a randomly generated pair of nodes
    n1 = randint(0, n-1)
    n2 = randint(0, n-1)
    X_train[m+i,0] = len(abstracts[n1]) + len(abstracts[n2])
    X_train[m+i,1] = abs(len(abstracts[n1]) - len(abstracts[n2]))
    X_train[m+i,2] = len(abstracts[n1].intersection(abstracts[n2]))
    X_train[m+i,3] = len(authors[n1]) + len(authors[n2])
    X_train[m+i,4] = abs(len(authors[n1]) - len(authors[n2]))
    X_train[m+i,5] = len(authors[n1].intersection(authors[n2]))
    X_train[m+i,6] = model[abstracts[edge[0]], abstracts[edge[1]]]
    y_train[m+i] = 0

TypeError: unhashable type: 'set'

In [ ]:
print('Size of training matrix:', X_train.shape)

Size of training matrix: (2183910, 7)


In [ ]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

In [ ]:
# Create the test matrix. Use the same 4 features as above
X_test = np.zeros((len(node_pairs), 7))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = len(abstracts[node_pair[0]]) + len(abstracts[node_pair[1]])
    X_test[i,1] = abs(len(abstracts[node_pair[0]]) - len(abstracts[node_pair[1]]))
    X_test[i,2] = len(abstracts[node_pair[0]].intersection(abstracts[node_pair[1]]))
    X_test[i,3] = len(authors[node_pair[0]]) + len(authors[node_pair[1]])
    X_test[i,4] = abs(len(authors[node_pair[0]]) - len(authors[node_pair[1]]))
    X_test[i,5] = len(authors[node_pair[0]].intersection(authors[node_pair[1]]))
    X_test[i,6] = model.wv.similarity(node_pair[0], node_pair[1])
print('Size of test matrix:', X_test.shape)

Size of test matrix: (106692, 7)


In [ ]:
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
# Use logistic regression to predict if two nodes are linked by an edge
logreg = LogisticRegression(C=1e5, solver='lbfgs')
logreg.fit(X_train, y_train)


y_pred = logreg.predict_proba(X_test)
y_pred = y_pred[:,1]

C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
48 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\ilsto\AppData\Local\Programs\Python\P

In [ ]:
# Write predictions to a file
predictions = zip(range(len(y_pred)), y_pred)
with open("submission_with_cos_sim.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)